In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
try:
    import google.colab
    colab = True
except:
    colab = False

In [2]:
if colab is True:
    # Running in Google Colab
    # Clone the repo
    !git clone https://github.com/sicara/easy-few-shot-learning
    %cd easy-few-shot-learning
    !pip install .
else:
    # Run locally
    # Ensure working directory is the project's root
    # Make sure easyfsl is installed!
    !git clone https://github.com/sicara/easy-few-shot-learning
    %cd easy-few-shot-learning
    !pip install .

Cloning into 'easy-few-shot-learning'...
remote: Enumerating objects: 1064, done.
remote: Counting objects: 100% (322/322), done.
remote: Compressing objects: 100% (176/176), done.
remote: Total 1064 (delta 209), reused 152 (delta 146), pack-reused 742
Receiving objects: 100% (1064/1064), 2.28 MiB | 3.00 MiB/s, done.
Resolving deltas: 100% (618/618), done.
/home/cse/FIneTune/PatternRecognition/easy-few-shot-learning
Processing /home/cse/FIneTune/PatternRecognition/easy-few-shot-learning
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for easyfsl: filename=easyfsl-1.3.0-py3-none-any.whl size=52364 sha256=c3a5f6d6ab66632909e5ba5311793148d85b16df395d134419c3e08173b15d92
  Stored in directory: /home/cse/.cache/pip/wheels/20/1e/11/d1cfe6477a5be5cbf2bc4c3a0c04d0c294067fb5646e7ac815
Successfully built easyfsl
  Attempting uninstall: easyfsl
    Found existing installation: easyfsl 1.3.0
    

In [4]:
from pathlib import Path
import random
from statistics import mean

import numpy as np
import torch
from torch import nn
from tqdm import tqdm
import torchvision
import torch.utils.data

In [5]:
random_seed = 0
np.random.seed(random_seed)
torch.manual_seed(random_seed)
random.seed(random_seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [22]:
image_size = 28

# Instantiate the datasets
train_set = torchvision.datasets.GTSRB(
    root='/home/cse/FIneTune/PatternRecognition/data',
    split='train',
    transform=torchvision.transforms.Compose(
        [
            torchvision.transforms.ToTensor(),
            torchvision.transforms.Resize([image_size,image_size]),
        ]
    ),
    target_transform=None,
    download=True)

  0%|          | 0/187490228 [00:00<?, ?it/s]

Extracting /home/cse/FIneTune/PatternRecognition/data/gtsrb/GTSRB-Training_fixed.zip to /home/cse/FIneTune/PatternRecognition/data/gtsrb


In [23]:
!mkdir /home/cse/FIneTune/PatternRecognition/data/gtsrb/GTSRB/Testing

!mv /home/cse/FIneTune/PatternRecognition/data/gtsrb/GTSRB/Training/00021 /home/cse/FIneTune/PatternRecognition/data/gtsrb/GTSRB/Testing
!mv /home/cse/FIneTune/PatternRecognition/data/gtsrb/GTSRB/Training/00037 /home/cse/FIneTune/PatternRecognition/data/gtsrb/GTSRB/Testing
!mv /home/cse/FIneTune/PatternRecognition/data/gtsrb/GTSRB/Training/00042 /home/cse/FIneTune/PatternRecognition/data/gtsrb/GTSRB/Testing
!mv /home/cse/FIneTune/PatternRecognition/data/gtsrb/GTSRB/Training/00016 /home/cse/FIneTune/PatternRecognition/data/gtsrb/GTSRB/Testing
!mv /home/cse/FIneTune/PatternRecognition/data/gtsrb/GTSRB/Training/00006 /home/cse/FIneTune/PatternRecognition/data/gtsrb/GTSRB/Testing
!mv /home/cse/FIneTune/PatternRecognition/data/gtsrb/GTSRB/Training/00005 /home/cse/FIneTune/PatternRecognition/data/gtsrb/GTSRB/Testing
!mv /home/cse/FIneTune/PatternRecognition/data/gtsrb/GTSRB/Training/00004 /home/cse/FIneTune/PatternRecognition/data/gtsrb/GTSRB/Testing
!mv /home/cse/FIneTune/PatternRecognition/data/gtsrb/GTSRB/Training/00035 /home/cse/FIneTune/PatternRecognition/data/gtsrb/GTSRB/Testing
!mv /home/cse/FIneTune/PatternRecognition/data/gtsrb/GTSRB/Training/00000 /home/cse/FIneTune/PatternRecognition/data/gtsrb/GTSRB/Testing
!mv /home/cse/FIneTune/PatternRecognition/data/gtsrb/GTSRB/Training/00029 /home/cse/FIneTune/PatternRecognition/data/gtsrb/GTSRB/Testing
!mv /home/cse/FIneTune/PatternRecognition/data/gtsrb/GTSRB/Training/00010 /home/cse/FIneTune/PatternRecognition/data/gtsrb/GTSRB/Testing
!mv /home/cse/FIneTune/PatternRecognition/data/gtsrb/GTSRB/Training/00012 /home/cse/FIneTune/PatternRecognition/data/gtsrb/GTSRB/Testing
!mv /home/cse/FIneTune/PatternRecognition/data/gtsrb/GTSRB/Training/00040 /home/cse/FIneTune/PatternRecognition/data/gtsrb/GTSRB/Testing
!mv /home/cse/FIneTune/PatternRecognition/data/gtsrb/GTSRB/Training/00009 /home/cse/FIneTune/PatternRecognition/data/gtsrb/GTSRB/Testing
!mv /home/cse/FIneTune/PatternRecognition/data/gtsrb/GTSRB/Training/00036 /home/cse/FIneTune/PatternRecognition/data/gtsrb/GTSRB/Testing
!mv /home/cse/FIneTune/PatternRecognition/data/gtsrb/GTSRB/Training/00031 /home/cse/FIneTune/PatternRecognition/data/gtsrb/GTSRB/Testing
!mv /home/cse/FIneTune/PatternRecognition/data/gtsrb/GTSRB/Training/00041 /home/cse/FIneTune/PatternRecognition/data/gtsrb/GTSRB/Testing
!mv /home/cse/FIneTune/PatternRecognition/data/gtsrb/GTSRB/Training/00018 /home/cse/FIneTune/PatternRecognition/data/gtsrb/GTSRB/Testing

In [8]:
import os

old_folder_name = '/home/cse/FIneTune/PatternRecognition/data/gtsrb/GTSRB/Training'
new_folder_name = '/home/cse/FIneTune/PatternRecognition/data/gtsrb/GTSRB/Trainingg'

# Rename the folder
os.rename(old_folder_name, new_folder_name)

old_folder_name = '/home/cse/FIneTune/PatternRecognition/data/gtsrb/GTSRB/Testing'
new_folder_name = '/home/cse/FIneTune/PatternRecognition/data/gtsrb/GTSRB/Training'

# Rename the folder
os.rename(old_folder_name, new_folder_name)

In [17]:
from pathlib import Path
import random
from statistics import mean

import numpy as np
import torch
from torch import nn
from tqdm import tqdm

import torch
import numpy as np
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import GTSRB
from torchvision.models import resnet18
from tqdm import tqdm

from easyfsl.samplers import TaskSampler
from easyfsl.utils import plot_images, sliding_average

from keras.preprocessing.image import ImageDataGenerator
import keras
from urllib.request import urlretrieve
import zipfile
import time
import matplotlib.pyplot as plt
import numpy as np

2023-05-21 04:33:37.256991: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-21 04:33:37.342584: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-21 04:33:37.696120: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-05-21 04:33:37.696157: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not l

In [24]:
import shutil
folder_path = "/home/cse/FIneTune/PatternRecognition/data/gtsrb/GTSRB/Training"
shutil.rmtree(folder_path)
# folder_path = "/content/data/gtsrb/GTSRB/Testing"
# shutil.rmtree(folder_path)

In [52]:
extract_path = "/home/cse/FIneTune/PatternRecognition/data/gtsrb/GTSRB"
validation_data_dir = '/home/cse/FIneTune/PatternRecognition/data/gtsrb/GTSRB/Training'

def train_data(train_url):
    zip_dir = "/home/cse/FIneTune/PatternRecognition/data/BelgiumTSC_Training.zip"

    print("Downloading Belgium TSC Training Dataset\n")
    urlretrieve(train_url, zip_dir)
    zip_ref = zipfile.ZipFile(zip_dir)

    print("Extracting Zip\n")
    zip_ref.extractall(extract_path)
    zip_ref.close()

def test_data(test_url):
    zip_dir = "/home/cse/FIneTune/PatternRecognition/data/BelgiumTSC_Training.zip"

    print("Downloading Belgium TSC Testing Dataset\n")
    urlretrieve(test_url, zip_dir)
    zip_ref = zipfile.ZipFile(zip_dir)

    print("Extracting Zip\n")
    zip_ref.extractall(extract_path)
    zip_ref.close()


def download_data():
    print("Download Datasets")
    start = time.time()
    train_data("http://btsd.ethz.ch/shareddata/BelgiumTSC/BelgiumTSC_Training.zip")
    # test_data("http://btsd.ethz.ch/shareddata/BelgiumTSC/BelgiumTSC_Testing.zip")
    end = time.time()
    print("Downloading Datasets 'BelgiumTSC' took ", end - start, 'seconds')

download_data()

Download Datasets

Extracting Zip



In [53]:
flag = 0
found = 0

In [57]:
for target_val in range(50, 120, 5):
  import os
  import shutil

  cntt =0
  summ = 0
  folder_A_path = '/home/cse/FIneTune/PatternRecognition/data/gtsrb/GTSRB/Training'

  # Iterate over subdirectories in folder_A
  for root, dirs, files in os.walk(folder_A_path):
      for dir in dirs:
          folder_B_path = os.path.join(root, dir)
          ppm_file_count = sum(1 for file in os.listdir(folder_B_path) if file.endswith('.ppm'))
          print(f"Folder {folder_B_path} contains {ppm_file_count} .ppm files.")

          summ += ppm_file_count
          print("VAL: ", target_val)
          if ppm_file_count < target_val:
            cntt += 1
            shutil.rmtree(folder_B_path)

  trains_set = GTSRB(
      root="/home/cse/FIneTune/PatternRecognition/data",
      # background=True,
      split="train",
      transform=transforms.Compose(
          [
              transforms.Grayscale(num_output_channels=3),
              transforms.RandomResizedCrop(image_size),
              transforms.RandomHorizontalFlip(),
              transforms.ToTensor(),
          ]
      ),
      download=False,
  )

  train_set_size = int(len(trains_set) * 0.8)
  val_set_size = len(trains_set) - train_set_size

  for vall in range(2):
    train_set, valid_set = torch.utils.data.random_split(trains_set, [train_set_size, val_set_size])
    train_set.get_labels = lambda: [instance[1] for instance in train_set]
    valid_set.get_labels = lambda: [instance[1] for instance in valid_set]

    train_dct = {}
    for item, label in enumerate(train_set.get_labels()):
      if label in train_dct:
          train_dct[label].append(item)
      else:
          train_dct[label] = [item]
          
    number_of_samples_per_label = [
                len(items_for_label) for items_for_label in train_dct.values()
            ]
    minimum_number_of_samples_per_label = min(number_of_samples_per_label)
    print("MIN: ", minimum_number_of_samples_per_label)
    label_with_minimum_number_of_samples = number_of_samples_per_label.index(
        minimum_number_of_samples_per_label
    )

    print("LABEL: ", label_with_minimum_number_of_samples)
    if n_shot + n_query < minimum_number_of_samples_per_label:
      flag = 1

    valid_dct = {}
    for item, label in enumerate(valid_set.get_labels()):
      if label in valid_dct:
          valid_dct[label].append(item)
      else:
          valid_dct[label] = [item]
          
    number_of_samples_per_label = [
                len(items_for_label) for items_for_label in valid_dct.values()
            ]
    minimum_number_of_samples_per_label = min(number_of_samples_per_label)
    print("MIN: ", minimum_number_of_samples_per_label)
    label_with_minimum_number_of_samples = number_of_samples_per_label.index(
        minimum_number_of_samples_per_label
    )

    print("LABEL: ", label_with_minimum_number_of_samples)
    if n_shot + n_query > minimum_number_of_samples_per_label:
      flag = 0
    
    print("Step 1: ", flag, found)
    if flag == 1:
      found = 1
      break
  print("Step 2: ", flag, found)
  if found == 1: break
  found = 0

Folder /home/cse/FIneTune/PatternRecognition/data/gtsrb/GTSRB/Training/00047 contains 147 .ppm files.
VAL:  50
Folder /home/cse/FIneTune/PatternRecognition/data/gtsrb/GTSRB/Training/00056 contains 95 .ppm files.
VAL:  50
Folder /home/cse/FIneTune/PatternRecognition/data/gtsrb/GTSRB/Training/00041 contains 148 .ppm files.
VAL:  50
Folder /home/cse/FIneTune/PatternRecognition/data/gtsrb/GTSRB/Training/00037 contains 98 .ppm files.
VAL:  50
Folder /home/cse/FIneTune/PatternRecognition/data/gtsrb/GTSRB/Training/00022 contains 375 .ppm files.
VAL:  50
Folder /home/cse/FIneTune/PatternRecognition/data/gtsrb/GTSRB/Training/00040 contains 242 .ppm files.
VAL:  50
Folder /home/cse/FIneTune/PatternRecognition/data/gtsrb/GTSRB/Training/00054 contains 118 .ppm files.
VAL:  50
Folder /home/cse/FIneTune/PatternRecognition/data/gtsrb/GTSRB/Training/00057 contains 78 .ppm files.
VAL:  50
Folder /home/cse/FIneTune/PatternRecognition/data/gtsrb/GTSRB/Training/00013 contains 90 .ppm files.
VAL:  50
Folde

Folder /home/cse/FIneTune/PatternRecognition/data/gtsrb/GTSRB/Training/00017 contains 79 .ppm files.
VAL:  65
Folder /home/cse/FIneTune/PatternRecognition/data/gtsrb/GTSRB/Training/00019 contains 231 .ppm files.
VAL:  65
Folder /home/cse/FIneTune/PatternRecognition/data/gtsrb/GTSRB/Training/00018 contains 81 .ppm files.
VAL:  65
Folder /home/cse/FIneTune/PatternRecognition/data/gtsrb/GTSRB/Training/00045 contains 74 .ppm files.
VAL:  65
Folder /home/cse/FIneTune/PatternRecognition/data/gtsrb/GTSRB/Training/00031 contains 63 .ppm files.
VAL:  65
Folder /home/cse/FIneTune/PatternRecognition/data/gtsrb/GTSRB/Training/00053 contains 199 .ppm files.
VAL:  65
Folder /home/cse/FIneTune/PatternRecognition/data/gtsrb/GTSRB/Training/00001 contains 110 .ppm files.
VAL:  65
MIN:  59
LABEL:  10
MIN:  13
LABEL:  14
Step 1:  0 0
MIN:  58
LABEL:  7
MIN:  13
LABEL:  18
Step 1:  0 0
Step 2:  0 0
Folder /home/cse/FIneTune/PatternRecognition/data/gtsrb/GTSRB/Training/00047 contains 147 .ppm files.
VAL:  7

In [58]:
target_val

75

In [59]:
val_set = valid_set

In [28]:
# image_size = 28

# # Instantiate the datasets
# train_set = torchvision.datasets.GTSRB(
#     root='/home/cse/FIneTune/PatternRecognition/data',
#     split='train',
#     transform=torchvision.transforms.Compose(
#         [
#             torchvision.transforms.ToTensor(),
#             torchvision.transforms.Resize([image_size,image_size]),
#         ]
#     ),
#     target_transform=None,
#     download=False)

# train_set_size = int(len(train_set) * 0.8)
# val_set_size = len(train_set) - train_set_size
# train_set, val_set = torch.utils.data.random_split(train_set, [train_set_size, val_set_size])

# train_set.get_labels = lambda: [
#     instance[1] for instance in train_set
# ]

# val_set.get_labels = lambda: [
#     instance[1] for instance in val_set
# ]

In [60]:
from torch.utils.data import DataLoader

batch_size = 128
n_workers = 0

# train_set = CUB(split="train", training=True)
train_loader = DataLoader(
    train_set,
    batch_size=batch_size,
    num_workers=n_workers,
    pin_memory=True,
    shuffle=True,
)

In [61]:
from easyfsl.modules import resnet12

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

model = resnet12(
    use_fc=True,
    num_classes=len(set(train_set.get_labels())),
).to(DEVICE)

In [62]:
from easyfsl.methods import PrototypicalNetworks,Finetune
from easyfsl.samplers import TaskSampler

n_way = 5
n_shot = 5
n_query = 10
n_validation_tasks = 500

# val_set = CUB(split="val", training=False)
val_sampler = TaskSampler(
    val_set, n_way=n_way, n_shot=n_shot, n_query=n_query, n_tasks=n_validation_tasks
)
val_loader = DataLoader(
    val_set,
    batch_sampler=val_sampler,
    num_workers=n_workers,
    pin_memory=True,
    collate_fn=val_sampler.episodic_collate_fn,
)

few_shot_classifier = Finetune(model).to(DEVICE)

In [63]:
from torch.optim import SGD, Optimizer
from torch.optim.lr_scheduler import MultiStepLR
from torch.utils.tensorboard import SummaryWriter


LOSS_FUNCTION = nn.CrossEntropyLoss()

n_epochs = 200
scheduler_milestones = [150, 180]
scheduler_gamma = 0.1
learning_rate = 1e-01
tb_logs_dir = Path(".")

train_optimizer = SGD(
    model.parameters(), lr=learning_rate, momentum=0.9, weight_decay=5e-4
)
train_scheduler = MultiStepLR(
    train_optimizer,
    milestones=scheduler_milestones,
    gamma=scheduler_gamma,
)

tb_writer = SummaryWriter(log_dir=str(tb_logs_dir))

In [73]:
def training_epoch(model_: nn.Module, data_loader: DataLoader, optimizer: Optimizer):
    all_loss = []
    model_.train()
    with tqdm(data_loader, total=len(data_loader), desc="Training") as tqdm_train:
        for images, labels in tqdm_train:
            optimizer.zero_grad()

            loss = LOSS_FUNCTION(model_(images.to(DEVICE)), labels.to(DEVICE))
            loss.backward()
            optimizer.step()

            all_loss.append(loss.item())

            tqdm_train.set_postfix(loss=mean(all_loss))

    return mean(all_loss)

In [74]:
from easyfsl.methods.tim import FewShotClassifier
from easyfsl.utils import evaluate


best_state = model.state_dict()
best_validation_accuracy = 0.0
validation_frequency = 10
for epoch in range(n_epochs):
    print(f"Epoch {epoch}")
    few_shot_classifier.backbone.requires_grad_(True)
    average_loss = training_epoch(model, train_loader, train_optimizer)

    if epoch % validation_frequency == validation_frequency - 1:

        # We use this very convenient method from EasyFSL's ResNet to specify
        # that the model shouldn't use its last fully connected layer during validation.
        model.set_use_fc(False)
        few_shot_classifier.backbone.requires_grad_(False)
        validation_accuracy = evaluate(
            few_shot_classifier, val_loader, device=DEVICE, tqdm_prefix="Validation"
        )
        model.set_use_fc(True)
        few_shot_classifier.backbone.requires_grad_(True)

        if validation_accuracy > best_validation_accuracy:
            best_validation_accuracy = validation_accuracy
            best_state = model.state_dict()
            print("Ding ding ding! We found a new best model!")

        tb_writer.add_scalar("Val/acc", validation_accuracy, epoch)
        # print(validation_accuracy)

    tb_writer.add_scalar("Train/loss", average_loss, epoch)

    # Warn the scheduler that we did an epoch
    # so it knows when to decrease the learning rate
    train_scheduler.step()
    if epoch % 10 == 0 : torch.save(best_state, '/home/cse/FIneTune/PatternRecognition/Finetune_5_5.pth')

Epoch 0


Training: 100%|██████████████████████| 22/22 [00:02<00:00, 10.59it/s, loss=4.37]


Epoch 1


Training: 100%|██████████████████████| 22/22 [00:01<00:00, 11.05it/s, loss=2.07]


Epoch 2


...

Training: 100%|████████████████████| 22/22 [00:01<00:00, 19.77it/s, loss=0.0555]


Epoch 198


Training: 100%|████████████████████| 22/22 [00:01<00:00, 19.78it/s, loss=0.0655]


Epoch 199


Validation: 100%|█████████████| 500/500 [00:14<00:00, 35.16it/s, accuracy=0.985]


In [75]:
model.load_state_dict(torch.load('/home/cse/FIneTune/PatternRecognition/Finetune_5_5.pth'))

<All keys matched successfully>

**Evaluation**

In [76]:
import os

old_folder_name = '/home/cse/FIneTune/PatternRecognition/data/gtsrb/GTSRB/Training'
new_folder_name = '/home/cse/FIneTune/PatternRecognition/data/gtsrb/GTSRB/Trainingg'

# Rename the folder
os.rename(old_folder_name, new_folder_name)

old_folder_name = '/home/cse/FIneTune/PatternRecognition/data/gtsrb/GTSRB/Testing'
new_folder_name = '/home/cse/FIneTune/PatternRecognition/data/gtsrb/GTSRB/Training'

# Rename the folder
os.rename(old_folder_name, new_folder_name)

In [77]:
n_test_tasks = 1000

# test_set = CUB(split="test", training=False)

test_set = torchvision.datasets.GTSRB(
    root='/home/cse/FIneTune/PatternRecognition/data',
    split='test',
    transform=torchvision.transforms.Compose(
        [
            torchvision.transforms.ToTensor(),
            torchvision.transforms.Resize([image_size,image_size]),
        ]
    ),
    target_transform=None,
    download=True
    )

test_set.get_labels = lambda: [
    instance[1] for instance in test_set
]

test_sampler = TaskSampler(
    test_set, n_way=n_way, n_shot=n_shot, n_query=n_query, n_tasks=n_test_tasks
)
test_loader = DataLoader(
    test_set,
    batch_sampler=test_sampler,
    num_workers=n_workers,
    pin_memory=True,
    collate_fn=test_sampler.episodic_collate_fn,
)

  0%|          | 0/88978620 [00:00<?, ?it/s]

Extracting /home/cse/FIneTune/PatternRecognition/data/gtsrb/GTSRB_Final_Test_Images.zip to /home/cse/FIneTune/PatternRecognition/data/gtsrb


  0%|          | 0/99620 [00:00<?, ?it/s]

Extracting /home/cse/FIneTune/PatternRecognition/data/gtsrb/GTSRB_Final_Test_GT.zip to /home/cse/FIneTune/PatternRecognition/data/gtsrb


In [78]:
model.set_use_fc(False)

accuracy = evaluate(few_shot_classifier, test_loader, device=DEVICE)
print(f"Average accuracy : {(100 * accuracy):.2f} %")

100%|███████████████████████| 1000/1000 [00:18<00:00, 54.26it/s, accuracy=0.798]

Average accuracy : 79.82 %


In [79]:
n_test_tasks = 10000

# test_set = CUB(split="test", training=False)

test_set = torchvision.datasets.GTSRB(
    root='/home/cse/FIneTune/PatternRecognition/data',
    split='test',
    transform=torchvision.transforms.Compose(
        [
            torchvision.transforms.ToTensor(),
            torchvision.transforms.Resize([image_size,image_size]),
        ]
    ),
    target_transform=None,
    download=True
    )

test_set.get_labels = lambda: [
    instance[1] for instance in test_set
]

test_sampler = TaskSampler(
    test_set, n_way=n_way, n_shot=n_shot, n_query=n_query, n_tasks=n_test_tasks
)
test_loader = DataLoader(
    test_set,
    batch_sampler=test_sampler,
    num_workers=n_workers,
    pin_memory=True,
    collate_fn=test_sampler.episodic_collate_fn,
)

In [80]:
model.set_use_fc(False)

accuracy = evaluate(few_shot_classifier, test_loader, device=DEVICE)
print(f"Average accuracy : {(100 * accuracy):.2f} %")

100%|█████████████████████| 10000/10000 [03:12<00:00, 51.92it/s, accuracy=0.796]

Average accuracy : 79.64 %
